## Python

In [115]:
%%python
from timeit import timeit
import numpy as np

def matmul_python(C, A, B):
    for m in range(C.rows):
        for k in range(A.cols):
            for n in range(C.cols):
                C[m, n] += A[m, k] * B[k, n]

class Matrix:
    def __init__(self, value, rows, cols):
        self.value = value
        self.rows = rows
        self.cols = cols

    def __getitem__(self, idxs):
        return self.value[idxs[0]][idxs[1]]

    def __setitem__(self, idxs, value):
        self.value[idxs[0]][idxs[1]] = value

def benchmark_matmul_python(M, N, K):
    A = Matrix(list(np.random.rand(M, K)), M, K)
    B = Matrix(list(np.random.rand(K, N)), K, N)
    C = Matrix(list(np.zeros((M, N))), M, N)
    secs = timeit(lambda: matmul_python(C, A, B), number=2)/2
    gflops = ((2*M*N*K)/secs) / 1e9
    print(gflops, "GFLOP/s")
    return gflops

In [116]:
python_gflops = benchmark_matmul_python(128, 128, 128).to_float64()

0.002040146805025923 GFLOP/s


# Mojo

## Matrix struct

In [117]:
import benchmark
from memory import memset_zero
from random import rand, random_float64

In [118]:
alias type = DType.float32

struct Matrix[rows: Int, cols: Int]:
    var data: DTypePointer[type]

    # Initialize zeroeing all values
    fn __init__(inout self):
        self.data = DTypePointer[type].alloc(rows * cols)
        memset_zero(self.data, rows * cols)

    # Initialize taking a pointer, don't set any elements
    fn __init__(inout self, data: DTypePointer[type]):
        self.data = data

    fn __init__(inout self, vec: SIMD[type, rows * cols]):
        self.data = DTypePointer[type].alloc(rows * cols)
        self.store(0, 0, vec)

    # Initialize with random values
    @staticmethod
    fn rand() -> Self:
        var data = DTypePointer[type].alloc(rows * cols)
        rand(data, rows * cols)
        return Self(data)
        
    fn __getitem__(self, y: Int, x: Int) -> Scalar[type]:
        return self.load[1](y, x)

    fn __setitem__(self, y: Int, x: Int, val: Scalar[type]):
        self.store[1](y, x, val)

    fn load[nelts: Int](self, y: Int, x: Int) -> SIMD[type, nelts]:
        return self.data.load[width=nelts](y * self.cols + x)

    fn store[nelts: Int](self, y: Int, x: Int, val: SIMD[type, nelts]):
        return self.data.store[width=nelts](y * self.cols + x, val)

    fn __str__(self) -> String:
        var matrix_string = String("")
        for r in range(rows):
            matrix_string += String(self.load[cols](r,0))
            matrix_string += "\n"
        return matrix_string

In [119]:
# Note that C, A, and B have types.
fn matmul_naive(C: Matrix, A: Matrix, B: Matrix):
    for m in range(C.rows):
        for k in range(A.cols):
            for n in range(C.cols):
                C[m, n] += A[m, k] * B[k, n]

In [131]:
# Parallelize the code by using the builtin parallelize function

alias nelts = simdwidthof[DType.float32]() * 2
from algorithm import vectorize, parallelize

fn matmul(C: Matrix, A: Matrix, B: Matrix):
    @parameter
    fn calc_row(m: Int):
        for k in range(A.cols):
            @parameter
            fn dot[nelts : Int](n : Int):
                C.store[nelts](m,n, C.load[nelts](m,n) + A[m,k] * B.load[nelts](k,n))
            vectorize[dot, nelts, size = C.cols]()
    parallelize[calc_row](C.rows, C.rows)

In [132]:
var A = Matrix[4, 2]()
A[1, 2] = 4.0
A[2, 1] = 3.0
print(A)

[0.0, 0.0]
[0.0, 0.0]
[4.0, 3.0]
[0.0, 0.0]



In [133]:
var B = Matrix[4, 2]()
B[1, 1] = 3.0
B[2, 1] = 4.0
print(B)

[0.0, 0.0]
[0.0, 3.0]
[0.0, 4.0]
[0.0, 0.0]



In [134]:
var C = Matrix[4, 2](SIMD[type, 8](0, 1, 2, 3, 4, 5))
print(C)

[0.0, 1.0]
[2.0, 3.0]
[4.0, 5.0]
[0.0, 0.0]



In [135]:
matmul(C, A, B)
print(C)

[0.0, 1.0]
[2.0, 3.0]
[4.0, 14.0]
[0.0, 0.0]



In [136]:
C.store[8](0, 0, SIMD[type, 8](0, 1, 2, 3, 4, 5))
print(C)

[0.0, 1.0]
[2.0, 3.0]
[4.0, 5.0]
[0.0, 0.0]



## Neural net

In [145]:
var inputs = Matrix[1, 4](SIMD[type, 4](1, 2, 3, 4))
print(inputs)

[1.0, 2.0, 3.0, 4.0]



In [146]:
var weights = Matrix[4, 1](SIMD[type, 4](1, 1, 2, 2))
print(weights)

1.0
1.0
2.0
2.0



In [147]:
var outputs = Matrix[1, 1]()
print(outputs)

0.0



In [148]:
matmul(outputs, inputs, weights)

In [149]:
print(outputs)

17.0

